<a href="https://colab.research.google.com/github/HARASEON/Deep_Learning_tensorflow/blob/main/Day3_transfer_learning_German_Traffic_signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**German Traffic signs**
* class: 43
* data length: 50,000
* size ranging: 15 '* 15 to 250 '* 250 pixels.
* val_loss < 0.5

In [1]:
from tensorflow.python.framework.ops import cpp_shape_inference_pb2
import urllib
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def download_and_extract_data():
    url = "https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip"
    urllib.request.urlretrieve(url, 'germantrafficsigns.zip')

    with zipfile.ZipFile('germantrafficsigns.zip','r') as zip_ref:
        zip_ref.extractall()

def preprocess(image, label):
  image /= 255
  return image, label


def solution_model():
  # Downloads and extracts the dataset to the directory that
  # contains this file.
    download_and_extract_data()
    BATCH_SIZE = 32
    IMG_SIZE = 30
  # The following code reads the training and validation data from their
  # respective directories, resizes them into the specified image size
  # and splits them into batches. You must fill in the image_size
  # argument for both training and validation data.
  # HINT: Image size is a tuple
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	directory='train/',
	label_mode='categorical',
	image_size=(IMG_SIZE, IMG_SIZE),
	batch_size=BATCH_SIZE)
    
    val_ds = tf.keras.preprocessing.image_dataset_from_directory (
 	directory='validation/',
	label_mode='categorical',
	image_size=(IMG_SIZE, IMG_SIZE),
	batch_size=BATCH_SIZE)
    
  # Normalizes train and validation datasets using the
  # preprocess() function.
  # Also makes other calls, as evident from the code, to prepare them for
  # training.
  # Do not batch or resize the images in the dataset here since it’s already
  # been done previously.
    train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Code to define the model
    model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dropout(0.25),
        Dense(128, activation='relu'),
        Dense(43, activation='softmax')
    ])
  # Code to compile and train the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    checkpoint_path = 'tmp_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                 save_weights_only=True,
                 save_best_only=True,
                 monitor='val_loss',
                 verbose=1)
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50,
        callbacks=[checkpoint, early_stopping]
    )
    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
  model = solution_model()
  model.save('german_traffic_sign-cnn.h5')

Found 31367 files belonging to 43 classes.
Found 7842 files belonging to 43 classes.
Epoch 1/50
981/981 [==============================] - ETA: 0s - loss: 1.7774 - acc: 0.4880
Epoch 1: val_loss improved from inf to 0.57007, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 22s 11ms/step - loss: 1.7774 - acc: 0.4880 - val_loss: 0.5701 - val_acc: 0.8318
Epoch 2/50
974/981 [============================>.] - ETA: 0s - loss: 0.4733 - acc: 0.8541
Epoch 2: val_loss improved from 0.57007 to 0.20689, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 10s 10ms/step - loss: 0.4725 - acc: 0.8543 - val_loss: 0.2069 - val_acc: 0.9417
Epoch 3/50
977/981 [============================>.] - ETA: 0s - loss: 0.2577 - acc: 0.9221
Epoch 3: val_loss improved from 0.20689 to 0.14879, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 9s 9ms/step - loss: 0.2576 - acc: 0.9221 - val_loss: 0.1488 - val_acc: 0.9607
Epoch 4/50
971/

ChatGPT
* transfer_learning
* prompt:please code with "German Traffic signs" dataset from tensorflow_datasets. use the transfer learning to get under 0.5 val_loss.
* responds: Here, we've used the ImageDataGenerator class from Keras to perform real-time data augmentation on the images during training. We've also used the MobileNetV2 model from Keras applications as the base model and added some layers on top of it to make the final model. Finally, we've set the trainable property of the base model to False, so that only the added layers are trained during fine-tuning. With these changes, we should be able to get a validation loss under 0.5.


In [3]:
import urllib
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def download_and_extract_data():
    url = "https://storage.googleapis.com/download.tensorflow.org/data/certificate/germantrafficsigns.zip"
    urllib.request.urlretrieve(url, 'germantrafficsigns.zip')

    with zipfile.ZipFile('germantrafficsigns.zip','r') as zip_ref:
        zip_ref.extractall()

def preprocess(image, label):
  image /= 255
  return image, label

def solution_model():
    # Downloads and extracts the dataset to the directory that
    # contains this file.
    download_and_extract_data()
    BATCH_SIZE = 32
    IMG_SIZE = 224

    # The following code reads the training and validation data from their
    # respective directories, resizes them into the specified image size
    # and splits them into batches.
    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    train_ds = train_datagen.flow_from_directory(
        directory='train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True,
        class_mode='categorical')

    val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    val_ds = val_datagen.flow_from_directory(
        directory='validation/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True,
        class_mode='categorical')

    # Code to define the model
    base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(43, activation='softmax')
    ])

    # Code to compile and train the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['acc']
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    checkpoint_path = 'tmp_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path,
                 save_weights_only=True,
                 save_best_only=True,
                 monitor='val_loss',
                 verbose=1)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs = 100,
        callbacks=[checkpoint, early_stopping]
    )

    model.load_weights(checkpoint_path)
    return model

if __name__ == '__main__':
  model = solution_model()
  model.save("german_traffic_sign_tr.h5")


Found 31367 images belonging to 43 classes.
Found 7842 images belonging to 43 classes.
Epoch 1/100
980/981 [============================>.] - ETA: 0s - loss: 2.3477 - acc: 0.3831
Epoch 1: val_loss improved from inf to 1.54909, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 68s 66ms/step - loss: 2.3477 - acc: 0.3830 - val_loss: 1.5491 - val_acc: 0.5199
Epoch 2/100
980/981 [============================>.] - ETA: 0s - loss: 1.3259 - acc: 0.5672
Epoch 2: val_loss improved from 1.54909 to 1.19036, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 62s 63ms/step - loss: 1.3257 - acc: 0.5673 - val_loss: 1.1904 - val_acc: 0.6019
Epoch 3/100
980/981 [============================>.] - ETA: 0s - loss: 0.9877 - acc: 0.6628
Epoch 3: val_loss improved from 1.19036 to 0.90530, saving model to tmp_checkpoint.ckpt
981/981 [==============================] - 70s 71ms/step - loss: 0.9875 - acc: 0.6628 - val_loss: 0.9053 - val_acc: 0.6912
Epoch 4/